# Lab 12: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Lab 9 imports
from funcs import *
from robotClass import *

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2022-05-20 04:20:38,035 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

# Data from robot runs

In [ ]:
# Data from multiple trials at each location
loc_0_3 = {
    0: [0.605, 0.605, 0.558, 0.554, 0.414, 0.414, 0.358, 0.358, 0.35, 0.35, 0.634, 0.634, 0.673, 0.673, 0.492, 0.492, 0.594, 0.594],
    1: [0.526, 0.526, 0.501, 0.597, 0.597, 0.471, 0.471, 0.376, 0.376, 0.47, 0.47, 0.671, 0.671, 0.444, 0.444, 0.657, 0.657, 0.576],
    2: [0.422, 0.422, 0.721, 0.683, 0.551, 0.404, 0.404, 0.388, 0.388, 0.398, 0.398, 0.664, 0.664, 0.77, 0.77, 0.779, 0.779, 0.8],
    3: [0.785, 0.802, 0.688, 0.5, 0.378, 0.378, 0.393, 0.393, 0.412, 0.412, 0.638, 0.638, 0.732, 0.732, 0.708, 0.708, 0.75, 0.75]
}

loc_5_3 = {
    0: [0.413, 0.413, 0.412, 0.458, 0.565, 0.565, 0.45, 0.45, 0.455, 0.455, 0.682, 0.682, 0.747, 0.747, 0.493, 0.493, 0.71, 0.71],
    1: [0.474, 0.417, 0.486, 0.56, 0.466, 0.466, 0.438, 0.438, 0.62, 0.62, 0.853, 0.853, 0.6, 0.6, 0.593, 0.593, 0.644, 0.644]
}

loc_5_minus3 = {
    0: [0.701, 0.701, 0.392, 0.444, 0.582, 0.609, 0.609, 0.598, 0.598, 0.596, 0.596, 0.504, 0.504, 0.69, 0.69, 0.433, 0.433, 0.486],
    1: [0.416, 0.408, 0.425, 0.517, 0.646, 0.646, 0.502, 0.502, 0.689, 0.689, 0.413, 0.413, 0.853, 0.853, 0.505, 0.505, 0.446, 0.446]
}

loc_minus3_minus2 = {
    0: [0.556, 0.435, 0.504, 0.617, 0.465, 0.472, 0.563, 0.563, 0.609, 0.609, 0.716, 0.716, 0.685, 0.685, 0.825, 0.825, 0.793, 0.793],
    1: [0.566, 0.386, 0.674, 0.479, 0.46, 0.63, 0.655, 0.655, 0.768, 0.768, 0.658, 0.658, 0.667, 0.667, 0.729, 0.729, 0.725, 0.725],
    2: [0.564, 0.564, 0.415, 0.608, 0.462, 0.556, 0.6, 0.6, 0.628, 0.756, 0.756, 0.673, 0.673, 0.667, 0.667, 0.806, 0.806, 0.766]
}

In [ ]:
n3n2_tof = [(655.0, 166.56714582443237), (723.0, 167.82818984985352), (919.0, 169.08666062355042), (719.0, 170.3430426120758), (659.0, 171.66160011291504), (681.0, 172.98369193077087), (863.0, 174.06581830978394), (858.0, 175.14470434188843), (696.0, 176.28691744804382), (712.0, 177.36669301986694), (849.0, 178.566876411438), (890.0, 179.58805203437805), (1221.0, 180.90769481658936), (3024.0, 182.16811084747314), (1919.0, 183.36635780334473), (2125.0, 184.56875205039978), (2162.0, 185.76617431640625), (671.0, 186.7842662334442), (673.0, 188.10858964920044)]

In [ ]:
y = [val[0] for val in n3n2_tof]
x = y[2:] + y[:1]
x

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble
        
        # Robot Control attribute
        self.rc = RobotControl(ble)

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, constant_data, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        #self.rc.pingRobot(clear = True) # clear attributes
        #self.rc.turn360(forwardSpeed = 140, backwardSpeed = 140, dir_ = 0)
        #await asyncio.sleep(5)
        
        #sensorVals = np.array(loc_0_3[1]).reshape(18, 1)
        #sensorVals = np.ones(18).reshape(18, 1)
        gyroVals = np.array([2974.44995117, 2974.45556641, 2993.55810547, 3012.73193359, 3031.74584961, 
                             3050.94506836, 3070.16479492, 3089.19018555, 3108.37109375, 3127.76220703,
                             3147.09936523, 3166.15039062, 3185.69360352, 3204.91992188, 3224.35595703,
                             3243.38232422, 3262.41137695, 3281.63598633]).reshape(18, 1)
        
        #sensorVals = np.array( [ val[0] / 1000 for val in self.rc.tof2_readings] ).reshape(18, 1)
        #gyroVals = np.array( [ val[0] for val in self.rc.imu_readings] ).reshape(18, 1)
        
        #print(sensorVals)
        #print(gyroVals)

        for i in range(18):
            self.rc.turn360(250, 95)
            await asyncio.sleep(1.5)
        
        #sensorVals = np.array(x).reshape(18, 1)/1000
        sensorVals = np.array(self.rc.tof_readings).reshape(18, 1)
        print(sensorVals)
        print(self.rc.tof_readings)
        
        # RETURN DISTANCES AND ANGLES
        return sensorVals, gyroVals

In [4]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-05-20 04:20:47,835 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:A5:6D:80:3C
2022-05-20 04:20:50,685 | INFO     |: Connected to C0:83:A5:6D:80:3C


In [5]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-20 04:20:59,684 | INFO     |:  | Number of observations per grid cell: 18
2022-05-20 04:20:59,685 | INFO     |:  | Precaching Views...


/Users/roninsharma/Desktop/Cornell/ECE 5960/Fast-Robots/Lab13/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-05-20 04:21:05,928 | INFO     |:  | Precaching Time: 6.242 secs
2022-05-20 04:21:05,929 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-20 04:21:05,929 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


### Lab 13

In [6]:
async def performLocalization(x1, y1): # x1, y1 are where we want to get to
    robot.rc.tof_readings = []
    
    # Reset Plots
    cmdr.reset_plotter()

    # Init Uniform Belief
    loc.init_grid_beliefs()

    # Get Observation Data by executing a 360 degree rotation motion
    await loc.get_observation_data()

    # Run Update Step
    loc.update_step()
    loc.plot_update_step_data(plot_data=True)
    print( [ loc.chris[0]*3, loc.chris[1]*3, loc.chris[2] ] )
    
    distance, angle = get_turndist(loc.chris[0]*3, loc.chris[1]*3, x1, y1, loc.chris[2])
    
    if (angle > 180):
        robot.rc.turn(180, 180, 0, 360-angle + 20, delta = 25)
    else:
        robot.rc.turn(180, 180, 0, angle + 20, delta = 25)
    print(f'angle: {angle}')
    
    await asyncio.sleep(2)
    
    robot.rc.pingRobot()
    latestTOF = robot.rc.tof_readings[-1] * 1000
    
    print(f'diff: {latestTOF - distance}')
    print(f'latest: {latestTOF}')
    print(f'distance: {distance}')
    
    y = latestTOF - distance
    if (y < 0):
        y *= -1
        
    robot.rc.updatePID(latestTOF - distance, 0.35, 0.75, PIDbuffer = 200)
    
    await asyncio.sleep(2)
    
    robot.rc.updatePID(latestTOF - distance, 0.35, 0.75, PIDbuffer = 200)
    
    await asyncio.sleep(2)
    
    robot.rc.updatePID(latestTOF - distance, 0.35, 0.75, PIDbuffer = 200)

In [7]:
def get_turndist(x0, y0, x1, y1, angle):
    delta_x = x1 - x0
    delta_y = y1 - y0
    dist = ( delta_x**2 + delta_y**2 ) ** 0.5
    theta = 180*np.arctan2(delta_y, delta_x)/np.pi
    print(theta)
    print(delta_x)
#     if delta_x < 0:
#         theta += 180
    theta -= angle
    theta = theta % 360
    
    print(dist)
    print(theta)
    return 300*dist/4, theta

get_turndist(0, 0, -1, 0, 0)

180.0
-1
1.0
180.0


(75.0, 180.0)

In [16]:
await performLocalization(2,-3)

2022-05-20 04:22:10,949 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-20 04:22:10,950 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.654]
 [0.496]
 [0.544]
 [0.379]
 [0.267]
 [0.245]
 [0.269]
 [0.357]
 [0.619]
 [2.117]
 [1.716]
 [0.738]
 [1.462]
 [3.257]
 [1.797]
 [0.963]
 [0.619]
 [0.519]]
[0.654, 0.496, 0.544, 0.379, 0.267, 0.245, 0.269, 0.357, 0.619, 2.117, 1.716, 0.738, 1.462, 3.257, 1.797, 0.963, 0.619, 0.519]
2022-05-20 04:22:38,519 | INFO     |: Update Step
2022-05-20 04:22:38,525 | INFO     |:      | Update Time: 0.005 secs
2022-05-20 04:22:38,527 | INFO     |: Bel index     : (11, 2, 4) with prob = 1.0
2022-05-20 04:22:38,528 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-20 04:22:38,532 | INFO     |: Belief        : (1.829, -0.610, -90.000)
[5.4864000000000015, -1.8287999999999998, -90.0]
-161.43105386301707
-3.4864000000000015
3.677865467903905
288.56894613698296
angle: 288.56894613698296
diff: 243.16008

* Got a correct localization from (-2,-1) to (0,0) -- video taken
* Also got a correct localization from (5,3) to (0,3) -- video taken

In [13]:
robot.rc.turn(160, 160, 1, 40, delta = 25)

In [15]:
robot.rc.updatePID(600, 0.35, 0.75, PIDbuffer = 200)

In [ ]:
FORWARD_SPEED = 180
BACKWARD_SPEED = 180

In [ ]:
# turn right 90 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 1, 90, delta = 50)

1 to 2

In [ ]:
# move forward
robot.rc.updatePID(300, 0.35, 0.75, PIDbuffer = 300)

In [ ]:
# turn right 90 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 1, 90, delta = 50)

2 to 3

In [ ]:
# move forward
robot.rc.updatePID(1800, 0.35, 0.75, PIDbuffer = 400)

In [ ]:
# turn right 60 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 1, 60, delta = 30)

3 to 4

In [ ]:
robot.rc.updatePID(400, 0.35, 0.75, PIDbuffer = 200)

In [ ]:
# turn left 60 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 0, 60, delta = 25)

4 to 5

In [ ]:
# move forward
robot.rc.updatePID(500, 0.35, 0.75, PIDbuffer = 200)

In [ ]:
# turn left 90 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 0, 90, delta = 35)

5 to 6 to 7

In [ ]:
# move forward
robot.rc.updatePID(600, 0.35, 0.75, PIDbuffer = 500)

In [ ]:
# turn left 90 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 0, 90, delta = 40)

7 to 8

In [ ]:
# move forward
robot.rc.updatePID(900, 0.35, 0.75, PIDbuffer = 200)

In [ ]:
# turn left 90 degrees
robot.rc.turn(FORWARD_SPEED, BACKWARD_SPEED, 0, 90, delta = 40)

In [ ]:
# LOCALIZATION

8 to 9

In [ ]:
# move forward
robot.rc.updatePID(1200, 0.35, 0.75, PIDbuffer = 200)

In [ ]:
# LOCALIZATION

In [ ]:
robot.rc.stop()

# Run an update step of the Bayes Filter

In [ ]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot GT
cmdr.plot_gt(0, 900/1000)

# Plot Odom and GT
#current_odom = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
#cmdr.plot_odom(current_odom[0], current_odom[1])

### Plotting

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title('Front ToF Sensor Readings')
plt.scatter([x for x in range(1, 19)], loc_0_3[1])
plt.xticks(np.arange(0, 20, 2))
plt.xlabel('Sensor Reading Number')
plt.ylabel('Sensor Value (m)')
plt.savefig('sensorReadings_0_3.png')

In [ ]:
plt.title('Front ToF Sensor Readings')
plt.scatter([x for x in range(1, 19)], loc_minus3_minus2[1])
plt.xticks(np.arange(0, 20, 2))
plt.xlabel('Sensor Reading Number')
plt.ylabel('Sensor Value (m)')
plt.savefig('sensorReadings_minus3_minus2.png')